## TEST and IMPORTS


In [2]:
import subprocess
import sys
import os
import time
import json
from IPython.display import display, HTML

print("🚀 Starting Environment Setup & Google Drive Test...")

# --- Configuration ---
GDRIVE_SECRET_NAME = "GOOGLE_DRIVE_API_KEY"
GDRIVE_TARGET_FOLDER_ID = '14YYjYqyE8b4VqlyKqXSvCoOrc6kDOCHS' # Your specific folder ID
PIP_PACKAGES = [
    "librosa", "soundfile", "torch", "transformers", "accelerate", "sentencepiece",
    "google-generativeai", "moviepy", "google-api-python-client",
    "google-auth-httplib2", "google-auth-oauthlib", "pydrive2"
]
CRYPTO_FIX_PACKAGES_UNINSTALL = ["pyOpenSSL", "cryptography"]
CRYPTO_FIX_PACKAGES_INSTALL = ["pyOpenSSL==23.2.0", "cryptography==41.0.7"] # Known good versions

# --- Helper to run pip commands ---
def run_pip(args):
    try:
        subprocess.check_call([sys.executable, "-m", "pip"] + args, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except subprocess.CalledProcessError as e:
        print(f"  ⚠️ Pip command failed: {' '.join(args)}. Error: {e}")
        return False

# --- 1. Install main packages ---
print("\n[1/4] 📦 Installing required packages...")
if not run_pip(["install"] + PIP_PACKAGES):
    print("  ❌ Failed to install one or more main packages. Please check errors above.")
    # Decide if you want to stop here or try to continue
else:
    print("  ✅ Main packages installed (or already present).")

# --- 2. Attempt Google Drive Authentication & Test ---
print("\n[2/4] 🔑 Authenticating and testing Google Drive access...")
gdrive_upload_enabled = False
drive_service = None

try:
    from kaggle_secrets import UserSecretsClient
    KAGGLE_SECRETS_AVAILABLE = True
except ImportError:
    KAGGLE_SECRETS_AVAILABLE = False
    print("  ℹ️ Kaggle Secrets client not found. GDrive auth might fail if key not in env.")

try:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    GDRIVE_LIBS_AVAILABLE = True
except ImportError:
    GDRIVE_LIBS_AVAILABLE = False
    print("  ❌ PyDrive2/google-auth libraries not found after pip install. This is unexpected.")

if GDRIVE_LIBS_AVAILABLE:
    service_creds_path = "/kaggle/working/temp_service_creds.json"
    try:
        if KAGGLE_SECRETS_AVAILABLE:
            secret_json_str = UserSecretsClient().get_secret(GDRIVE_SECRET_NAME)
            if not secret_json_str:
                raise ValueError(f"Kaggle Secret '{GDRIVE_SECRET_NAME}' not found.")
            with open(service_creds_path, "w") as f: f.write(secret_json_str)
            
            settings = {"client_config_backend": "service", "service_config": {"client_json_file_path": service_creds_path}}
            gauth = GoogleAuth(settings=settings)
            gauth.ServiceAuth()
            drive_service = GoogleDrive(gauth)
            print("  ✅ GDrive Authenticated.")

            # Test upload/delete
            dummy_fn = f"gdrive_test_{int(time.time())}.txt"
            local_dummy_fp = os.path.join("/kaggle/working/", dummy_fn)
            gdrive_dummy_id = None
            with open(local_dummy_fp, "w") as f: f.write("test")
            
            meta = {'title': dummy_fn}
            if GDRIVE_TARGET_FOLDER_ID: meta['parents'] = [{'id': GDRIVE_TARGET_FOLDER_ID}]
            
            gfile = drive_service.CreateFile(meta)
            gfile.SetContentFile(local_dummy_fp)
            gfile.Upload({'convert': False})
            gdrive_dummy_id = gfile['id']
            drive_service.CreateFile({'id': gdrive_dummy_id}).Delete()
            os.remove(local_dummy_fp)
            gdrive_upload_enabled = True
            print(f"  ✅ GDrive access test to folder '{GDRIVE_TARGET_FOLDER_ID or 'Root'}' PASSED.")

        else: # No Kaggle Secrets
            print("  ℹ️ Kaggle Secrets unavailable, GDrive auth relies on environment variables or pre-configuration not handled by this script.")
            # Potentially try to init GoogleAuth without explicit service creds if user has gcloud configured,
            # but that's less common in Kaggle and makes script less self-contained.
            # For now, we assume service account key via Kaggle Secrets is the primary method.

    except Exception as e:
        print(f"  ⚠️ GDrive authentication or test FAILED: {e}")
        gdrive_error_str = str(e).lower()
        # --- 3. Conditional Crypto Fix ---
        # Check for common crypto-related error signatures
        if "sign" in gdrive_error_str or "openssl" in gdrive_error_str or "cryptography" in gdrive_error_str or "'service_config'" in gdrive_error_str or "valid" in gdrive_error_str :
            print("\n[3/4] 🔧 Detected GDrive error potentially related to crypto libraries. Attempting fix...")
            print(f"  Uninstalling: {', '.join(CRYPTO_FIX_PACKAGES_UNINSTALL)}")
            run_pip(["uninstall", "-y"] + CRYPTO_FIX_PACKAGES_UNINSTALL)
            print(f"  Installing: {', '.join(CRYPTO_FIX_PACKAGES_INSTALL)}")
            if run_pip(["install"] + CRYPTO_FIX_PACKAGES_INSTALL):
                print("  ✅ Crypto libraries reinstalled.")
                print("  🔴🔴🔴 KERNEL RESTART REQUIRED! 🔴🔴🔴")
                print("  Please restart the kernel ('Run' > 'Restart Session' or 'Kernel' > 'Restart') and re-run this cell.")
                display(HTML("<h3 style='color:red;'>KERNEL RESTART REQUIRED to apply crypto fixes. Re-run this cell after restart.</h3>"))
            else:
                print("  ❌ Crypto fix failed during pip install. Manual intervention may be needed.")
        else: # GDrive error not obviously crypto-related
             print("\n[3/4] GDrive error does not seem to be a common crypto issue. No automatic fix attempted for this.")
    finally:
        if os.path.exists(service_creds_path): os.remove(service_creds_path)
        if 'local_dummy_fp' in locals() and os.path.exists(local_dummy_fp): os.remove(local_dummy_fp)
else: # GDRIVE_LIBS_AVAILABLE is False
    print("\n[2/4 & 3/4] ❌ PyDrive2/google-auth libraries failed to import. Cannot test GDrive or apply crypto fixes.")


# --- 4. Final Status ---
print("\n[4/4] ✨ Environment Setup Summary:")
if gdrive_upload_enabled:
    print("  ✅ Google Drive access VERIFIED and ENABLED.")
else:
    print("  ⚠️ Google Drive access NOT VERIFIED or FAILED. Uploads will be disabled in the main script.")
    print("     If a crypto fix was applied, ensure you have restarted the kernel and re-run this cell.")
    print("     If issues persist, check GDrive folder permissions, secret key, and network access.")

print("\nSetup cell finished. If kernel restart was indicated, DO IT NOW.")

🚀 Starting Environment Setup & Google Drive Test...

[1/4] 📦 Installing required packages...
  ✅ Main packages installed (or already present).

[2/4] 🔑 Authenticating and testing Google Drive access...
  ✅ GDrive Authenticated.
  ✅ GDrive access test to folder '14YYjYqyE8b4VqlyKqXSvCoOrc6kDOCHS' PASSED.

[4/4] ✨ Environment Setup Summary:
  ✅ Google Drive access VERIFIED and ENABLED.

Setup cell finished. If kernel restart was indicated, DO IT NOW.


## SCRIPT ---------------------------------------------

In [9]:
# ─── STEP 0: Imports & Setup ───────────────────────────────────────────────
import os
import time
import json
from IPython.display import display, HTML
import google.generativeai as genai

# For Google Drive
try:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    GDRIVE_AVAILABLE = True
except ImportError:
    GDRIVE_AVAILABLE = False
    print("⚠️ PyDrive2 or google-auth libraries not found. Google Drive upload will be skipped.")

# Kaggle Secrets client
KAGGLE_SECRETS_AVAILABLE = False
try:
    from kaggle_secrets import UserSecretsClient
    KAGGLE_SECRETS_AVAILABLE = True
except ImportError:
    print("ⓘ Kaggle Secrets client not found. API keys must be provided manually or via environment variables.")

# Supported file types for handouts
SUPPORTED_HANDOUT_EXTS = ('.md', '.pdf')
YOUR_GDRIVE_FOLDER_ID = '14YYjYqyE8b4VqlyKqXSvCoOrc6kDOCHS'

# ─── STEP 1: API Key and Google Drive Setup ────────────────────────────────
def get_api_key(secret_name="GEMINI_API_KEY", prompt_message="Gemini API Key") -> str:
    # (Unchanged)
    api_key = ""
    if KAGGLE_SECRETS_AVAILABLE:
        try:
            user_secrets = UserSecretsClient()
            api_key = user_secrets.get_secret(secret_name)
            if api_key:
                print(f"✅ {prompt_message} found in Kaggle Secrets.")
                return api_key
        except Exception as e:
            print(f"ⓘ Failed to get {prompt_message} from Kaggle secrets: {e}. Will check environment variables.")
    api_key = os.getenv(secret_name)
    if api_key:
        print(f"✅ {prompt_message} found in environment variables.")
        return api_key
    print(f"⚠️ {prompt_message} not found. Please enter it manually.")
    api_key = input(f"🔑 Please enter your {prompt_message}: ").strip()
    return api_key

gemini_api_key = get_api_key()
if not gemini_api_key:
    raise ValueError("A Gemini API Key is required to proceed.")
genai.configure(api_key=gemini_api_key)

system_instruction_italian = "Sei un assistente accademico che produce materiale di studio. Comunica e genera risposte esclusivamente in lingua italiana, mantenendo uno stile formale e chiaro."
gemini_model = genai.GenerativeModel(
    "models/gemini-2.5-flash-preview-05-20",
    system_instruction=system_instruction_italian
)
print(f"✅ Gemini API configured with model: '{gemini_model.model_name}' and a system instruction to enforce Italian output.")

# --- Google Drive and File Selection Setup (Unchanged and omitted for brevity) ---
def authenticate_gdrive():
    if not GDRIVE_AVAILABLE or not KAGGLE_SECRETS_AVAILABLE: return None
    service_account_creds_file = "/kaggle/working/service_creds.json"
    try:
        user_secrets = UserSecretsClient()
        secret_json_str = user_secrets.get_secret("GOOGLE_DRIVE_API_KEY")
        if not secret_json_str:
            print("⚠️ Google Drive service account key (GOOGLE_DRIVE_API_KEY) not found in Kaggle Secrets.")
            return None
        with open(service_account_creds_file, "w") as f: f.write(secret_json_str)
        settings = { "client_config_backend": "service", "service_config": { "client_json_file_path": service_account_creds_file, }}
        gauth = GoogleAuth(settings=settings)
        gauth.ServiceAuth()
        drive = GoogleDrive(gauth)
        print("✅ Authenticated with Google Drive using service account.")
        return drive
    except Exception as e:
        print(f"⚠️ Error authenticating with Google Drive: {e}")
        return None
    finally:
        if os.path.exists(service_account_creds_file):
            try: os.remove(service_account_creds_file)
            except Exception as e_clean: print(f"Warning: Could not remove temp GDrive creds file: {e_clean}")
drive_service = authenticate_gdrive()
gdrive_upload_enabled = False
def test_gdrive_access(drive, gdrive_folder_id_to_test):
    if not drive: return False
    print("\n🧪 Testing Google Drive access (upload/delete)...")
    dummy_filename = f"gdrive_access_test_{int(time.time())}.txt"
    local_dummy_filepath = os.path.join("/kaggle/working/", dummy_filename)
    gdrive_dummy_file_id = None
    try:
        with open(local_dummy_filepath, "w") as f: f.write("GDrive access test.")
        file_metadata = {'title': dummy_filename, 'parents': [{'id': gdrive_folder_id_to_test}]} if gdrive_folder_id_to_test else {'title': dummy_filename}
        gfile = drive.CreateFile(file_metadata)
        gfile.SetContentFile(local_dummy_filepath)
        gfile.Upload({'convert': False})
        gdrive_dummy_file_id = gfile['id']
        file_to_delete = drive.CreateFile({'id': gdrive_dummy_file_id})
        file_to_delete.Delete()
        print("✅ Google Drive access test PASSED.")
        return True
    except Exception as e:
        print(f"  - ⚠️ Google Drive access test FAILED: {e}")
        if gdrive_dummy_file_id:
            try:
                drive.CreateFile({'id': gdrive_dummy_file_id}).Delete()
            except Exception as e_cleanup:
                print(f"  - Warning: Could not clean up test file from GDrive: {e_cleanup}")
        return False
    finally:
        if os.path.exists(local_dummy_filepath):
            try: os.remove(local_dummy_filepath);
            except Exception as e_clean_local: print(f"  - Warning: Could not remove local test file: {e_clean_local}")
if drive_service:
    gdrive_upload_enabled = test_gdrive_access(drive_service, YOUR_GDRIVE_FOLDER_ID)
    if not gdrive_upload_enabled: print("🔴 Google Drive uploads will be disabled due to failed access test.")
else:
    print("ℹ️ Google Drive service not initialized. GDrive uploads will be disabled.")
handout_candidates = []
for root, _, files in os.walk('/kaggle/input'):
    for fn in files:
        if fn.lower().endswith(SUPPORTED_HANDOUT_EXTS):
            handout_candidates.append(os.path.join(root, fn))
if not handout_candidates: raise FileNotFoundError("No handout files (.md, .pdf) found in /kaggle/input.")
print("\n📚 Available Handout Files (.md, .pdf):")
for i, path in enumerate(handout_candidates): print(f" [{i}] {os.path.basename(path)}")
selected_indices_str = input(f"\nEnter indices of HANDOUTS to process (e.g., 0,2,3), or 'all': ")
selected_handouts = []
if selected_indices_str.strip().lower() == 'all': selected_handouts = handout_candidates
else:
    try:
        selected_indices = [int(idx.strip()) for idx in selected_indices_str.split(',')]
        for idx in selected_indices:
            if 0 <= idx < len(handout_candidates): selected_handouts.append(handout_candidates[idx])
            else: print(f"⚠️ Warning: Index {idx} is out of range. Skipping.")
    except ValueError: raise ValueError("Invalid input for handout selection.")
if not selected_handouts: raise ValueError("No handouts selected for processing.")
print("\n👍 Handouts selected for processing:")
for f_path in selected_handouts: print(f"  -> {os.path.basename(f_path)}")
pdf_candidates = [p for p in handout_candidates if p.lower().endswith('.pdf')]
if not pdf_candidates:
    for root, _, files in os.walk('/kaggle/input'):
      for fn in files:
        if fn.lower().endswith('.pdf') and os.path.join(root, fn) not in pdf_candidates: pdf_candidates.append(os.path.join(root, fn))
if not pdf_candidates: raise FileNotFoundError("No PDF files found in /kaggle/input to be used as a PREPARATION GUIDE.")
print("\n❓ Available PDF files to use as PREPARATION GUIDE:")
for i, path in enumerate(pdf_candidates): print(f" [{i}] {os.path.basename(path)}")
try:
    guide_idx = int(input(f"\nEnter the index of the PREPARATION GUIDE file (0-{len(pdf_candidates)-1}): "))
    if not (0 <= guide_idx < len(pdf_candidates)): raise ValueError("Index out of range.")
    preparation_guide_path = pdf_candidates[guide_idx]
    print(f"\n✅ Using '{os.path.basename(preparation_guide_path)}' as the Preparation Guide.")
except (ValueError, IndexError): raise ValueError("Invalid selection for the Preparation Guide.")
def upload_to_gdrive_if_enabled(drive, local_filepath, gdrive_folder_id_to_use, filename_on_gdrive=None):
    if not gdrive_upload_enabled or not drive: return None
    if not os.path.exists(local_filepath):
        display(HTML(f"<p style='color:red;'>⚠️ Local file for GDrive upload not found: {local_filepath}</p>")); return None
    try:
        actual_filename = filename_on_gdrive if filename_on_gdrive else os.path.basename(local_filepath)
        file_metadata = {'title': actual_filename, 'parents': [{'id': gdrive_folder_id_to_use}]} if gdrive_folder_id_to_use else {'title': actual_filename}
        gfile = drive.CreateFile(file_metadata)
        gfile.SetContentFile(local_filepath)
        gfile.Upload({'convert': False})
        display(HTML(f"<p style='color:blue;'>📤 File '{actual_filename}' uploaded to Google Drive. (ID: {gfile['id']})</p>"))
        return gfile['id']
    except Exception as e:
        display(HTML(f"<p style='color:red;'>⚠️ Error uploading {local_filepath} to Google Drive: {e}")); return None
def call_gemini_with_retry(model, prompt_list, max_retries=3, delay_seconds=20):
    retries = 0
    while retries < max_retries:
        try:
            response = model.generate_content(prompt_list)
            return response
        except Exception as e:
            retries += 1
            display(HTML(f"<p style='color:orange;'>⚠️ API call failed with error: {e}.<br>This is often due to rate limits (10 requests/minute). Retrying in {delay_seconds}s... (Attempt {retries}/{max_retries})</p>"))
            if retries >= max_retries:
                display(HTML("<p style='color:red; font-weight:bold;'>🔴 Max retries reached. Failing this step.</p>"))
                raise e
            time.sleep(delay_seconds)

# ─── STEP 3: Core Processing Loop (Enrich First, Summarize Second) ───────────────────
all_final_handouts = []
uploaded_prep_guide_file = None

try:
    print("\nCaricamento della Guida alla Preparazione su API Gemini...")
    uploaded_prep_guide_file = genai.upload_file(path=preparation_guide_path, display_name="GUIDA_ALLA_PREPARAZIONE.pdf")
    print("✔️ Guida alla Preparazione caricata.")

    for handout_path in selected_handouts:
        base_filename = os.path.splitext(os.path.basename(handout_path))[0]
        display(HTML(f"<hr><h3>📄 Elaborazione in corso: {os.path.basename(handout_path)}</h3>"))
        uploaded_handout_file = None
        
        try:
            # --- PHASE 1: ENRICH FULL DOCUMENT FIRST ---
            # ... (Logic is unchanged, this is the optimal path)
            display(HTML(f"<p>✨ <b>Fase 1: Arricchimento del documento completo '{os.path.basename(handout_path)}'...</b></p>"))
            uploaded_handout_file = genai.upload_file(path=handout_path, display_name=f"originale_{base_filename}")
            prompt_enrich = [
                f"""**OBIETTIVO:** Integrare il documento originale (`{uploaded_handout_file.display_name}`) per garantire che risponda pienamente alle domande pertinenti della guida alla preparazione (`{uploaded_prep_guide_file.display_name}`).\n\n**ISTRUZIONI:**\n1.  **Analizza i File:** Leggi attentamente il documento originale e la guida alla preparazione.\n2.  **Identifica Domande Pertinenti:** Estrai dalla guida solo le domande il cui argomento è trattato nel documento originale.\n3.  **Integra e Arricchisci:** Modifica il testo del documento originale, integrando in modo fluido e discorsivo tutte le informazioni necessarie per rispondere in modo esauriente alle domande pertinenti. NON limitarti ad aggiungere testo; se necessario, riformula parti esistenti per incorporare i nuovi concetti.\n4.  **Aggiungi Sezione Domande:** Al termine del documento arricchito, aggiungi una nuova sezione Markdown intitolata `## Domande Pertinenti Identificate in Questa Dispensa`. Sotto questo titolo, elenca puntualmente le domande che hai identificato e a cui il testo ora è in grado di rispondere.\n5.  **Output:** Il tuo output deve essere l'INTERO testo del documento originale, ora arricchito e con la sezione finale delle domande.\n \n**IMPORTANTE: L'intero output deve essere esclusivamente in lingua ITALIANA.**""",
                uploaded_handout_file, uploaded_prep_guide_file
            ]
            response_p1 = call_gemini_with_retry(gemini_model, prompt_enrich)
            enriched_full_text = response_p1.text
            display(HTML("<p style='color:green;'>✔️ Fase 1 completata. Documento completo arricchito.</p>"))

            # --- PHASE 2: SUMMARIZE THE ENRICHED DOCUMENT ---
            display(HTML(f"<p>🧐 <b>Fase 2: Sintesi finale del testo arricchito...</b></p>"))
            prompt_summarize = [
                f"""**OBIETTIVO:** Riassumere in modo conciso il testo arricchito fornito, che ora include le risposte a domande d'esame.\n\n**ISTRUZIONE IMPERATIVA:**\nLa priorità assoluta è la sintesi. Il testo finale deve avere una lunghezza MASSIMA di 5 pagine. Sii incisivo ed elimina ogni ridondanza non essenziale.\n\n**ISTRUZIONI DETTAGLIATE:**\n1.  **Sintesi Estrema:** Riduci il testo fornito alla lunghezza massima specificata (5 pagine).\n2.  **Mantieni i Concetti Chiave:** Pur essendo breve, il riassunto deve conservare tutti i concetti fondamentali e, soprattutto, le informazioni che permettono di rispondere alle domande elencate nella sezione finale "Domande Pertinenti".\n3.  **Conserva la Struttura:** Mantieni la formattazione Markdown, le intestazioni e la sezione finale `## Domande Pertinenti Identificate...` così com'è.\n4.  **Fedeltà:** Non introdurre concetti nuovi. La sintesi deve essere un distillato fedele del testo fornito.\n\n**IMPORTANTE: L'intero output deve essere esclusivamente in lingua ITALIANA.**""",
                enriched_full_text
            ]
            response_p2 = call_gemini_with_retry(gemini_model, prompt_summarize)
            final_summary = response_p2.text
            
            separator = f"\n\n\n---\n\n# {base_filename}\n\n---\n\n\n"
            all_final_handouts.append(final_summary + separator)
            display(HTML("<p style='color:green;'>✔️ Fase 2 completata. Sintesi finale generata.</p>"))

        except Exception as e:
            display(HTML(f"<p style='color:red; font-weight:bold;'>⚠️ Si è verificato un errore durante l'elaborazione di {os.path.basename(handout_path)}: {e}</p>"))
            all_final_handouts.append(f"# ERRORE DURANTE L'ELABORAZIONE: {os.path.basename(handout_path)}\n\nSi è verificato un errore: {e}\n")
        finally:
            if uploaded_handout_file:
                genai.delete_file(uploaded_handout_file.name)
finally:
    if uploaded_prep_guide_file:
        genai.delete_file(uploaded_prep_guide_file.name)
        print("\n✔️ File API di Gemini puliti.")

# ─── STEP 4: Final Compilation and Saving (Draft and Revised) ────────────────────────
if all_final_handouts:
    display(HTML("<hr><h3>💾 Compilazione e salvataggio dei documenti finali...</h3>"))
    
    # --- 4.1: Save the "Safe" Draft Version ---
    consolidated_draft = "".join(all_final_handouts)
    draft_md_path = "/kaggle/working/Sintesi_Consolidata_Bozza.md"
    
    try:
        with open(draft_md_path, "w", encoding="utf-8") as f:
            f.write(consolidated_draft)
        display(HTML(f"<p style='color:green; font-weight:bold;'>💾 Bozza consolidata (non revisionata) salvata localmente: <code>{draft_md_path}</code></p>"))
        upload_to_gdrive_if_enabled(drive_service, draft_md_path, YOUR_GDRIVE_FOLDER_ID)
    except Exception as e:
        display(HTML(f"<p style='color:red; font-weight:bold;'>⚠️ Errore durante il salvataggio della bozza: {e}</p>"))

    # --- 4.2: Attempt the Final Editing Pass to create the Revised Version ---
    display(HTML("<p>✍️ <b>Fase Finale: Tentativo di revisione editoriale per coesione e fluidità...</b></p>"))
    prompt_final_edit = [
        """**OBIETTIVO:** Agire come un redattore finale su un documento consolidato.
        
**CONTESTO:** Il testo che segue è una raccolta di diverse sintesi di dispense, concatenate insieme. La tua mansione è di trasformarlo in un capitolo unico, fluido e coeso.

**ISTRUZIONI:**
1.  **Migliora la Fluidità:** Rivedi le transizioni tra le diverse sezioni (separate da `--- # nomefile ---`). Rendi il passaggio da un argomento all'altro più naturale, aggiungendo brevi frasi di collegamento se necessario.
2.  **Elimina Ridondanze Palesi:** Se un concetto fondamentale viene definito in dettaglio in una sezione e poi di nuovo in un'altra, mantieni la definizione dettagliata la prima volta e sostituisci la seconda con un breve richiamo (es. "come abbiamo visto in precedenza..."). Non eliminare argomenti interi.
3.  **Armonizza lo Stile:** Assicurati che il tono e lo stile siano coerenti in tutto il documento.
4.  **Output Finale:** Restituisci l'intero documento finale, rivisto e migliorato, mantenendo la struttura dei titoli principali.

**IMPORTANTE: L'intero output deve essere esclusivamente in lingua ITALIANA.**""",
        consolidated_draft
    ]
    
    try:
        final_response = call_gemini_with_retry(gemini_model, prompt_final_edit)
        revised_content = final_response.text
        revised_md_path = "/kaggle/working/Capitolo_Unico_Revisionato.md"
        
        with open(revised_md_path, "w", encoding="utf-8") as f:
            f.write(revised_content)
        display(HTML(f"<p style='color:green; font-weight:bold;'>✅ Revisione finale completata con successo! Salvato in: <code>{revised_md_path}</code></p>"))
        upload_to_gdrive_if_enabled(drive_service, revised_md_path, YOUR_GDRIVE_FOLDER_ID)

    except Exception as e:
        display(HTML(f"<p style='color:red; font-weight:bold;'>⚠️ Errore durante la revisione finale: {e}</p>"))
        display(HTML("<p>ℹ️ La versione revisionata non è stata creata. La versione 'Bozza' è stata salvata correttamente.</p>"))

display(HTML("<hr><h3>✅ Tutti i file selezionati sono stati elaborati.</h3>"))
if gdrive_upload_enabled and drive_service :
    display(HTML(f"<p>📤 Controlla la tua cartella di Google Drive per i file caricati.</p>"))
elif drive_service and not gdrive_upload_enabled:
    display(HTML("<p>ℹ️ È stato tentato il caricamento su Google Drive ma il test di accesso preliminare è fallito. I file sono stati salvati localmente.</p>"))
else:
    display(HTML("<p>ℹ️ Google Drive non è configurato o l'autenticazione è fallita. I file sono stati salvati localmente.</p>"))

✅ Gemini API Key found in Kaggle Secrets.
✅ Gemini API configured with model: 'models/gemini-2.5-flash-preview-05-20' and a system instruction to enforce Italian output.
✅ Authenticated with Google Drive using service account.

🧪 Testing Google Drive access (upload/delete)...
✅ Google Drive access test PASSED.

📚 Available Handout Files (.md, .pdf):
 [0] GUIDA ALLA PREPARAZIONE Filosofia Morale 2024-2025.pdf
 [1] rec22_Pe8hcE_handout.md
 [2] rec28_Pe8hcE_handout.md
 [3] rec13_handout.md
 [4] rec18_handout.md
 [5] rec12_handout.md
 [6] rec26_Pe8hcE_handout.md
 [7] rec23_Pe8hcE_handout.md
 [8] rec15_handout.md
 [9] rec25_Pe8hcE_handout.md
 [10] rec17_handout.md
 [11] rec30_Pe8hcE_handout.md
 [12] rec29_Pe8hcE_handout.md
 [13] rec20_handout.md
 [14] rec27_handout.md
 [15] rec9_handout.md
 [16] rec24_handout.md
 [17] rec19_handout.md
 [18] rec10_handout.md
 [19] rec16_handout.md
 [20] rec11_handout.md
 [21] rec21_Pe8hcE_handout.md
 [22] rec14_handout.md



Enter indices of HANDOUTS to process (e.g., 0,2,3), or 'all':  15,18,20,5,3,22,8,19,10,4,17,13,21,1,7,16,9,6,14,2,12,11



👍 Handouts selected for processing:
  -> rec9_handout.md
  -> rec10_handout.md
  -> rec11_handout.md
  -> rec12_handout.md
  -> rec13_handout.md
  -> rec14_handout.md
  -> rec15_handout.md
  -> rec16_handout.md
  -> rec17_handout.md
  -> rec18_handout.md
  -> rec19_handout.md
  -> rec20_handout.md
  -> rec21_Pe8hcE_handout.md
  -> rec22_Pe8hcE_handout.md
  -> rec23_Pe8hcE_handout.md
  -> rec24_handout.md
  -> rec25_Pe8hcE_handout.md
  -> rec26_Pe8hcE_handout.md
  -> rec27_handout.md
  -> rec28_Pe8hcE_handout.md
  -> rec29_Pe8hcE_handout.md
  -> rec30_Pe8hcE_handout.md

❓ Available PDF files to use as PREPARATION GUIDE:
 [0] GUIDA ALLA PREPARAZIONE Filosofia Morale 2024-2025.pdf



Enter the index of the PREPARATION GUIDE file (0-0):  0



✅ Using 'GUIDA ALLA PREPARAZIONE Filosofia Morale 2024-2025.pdf' as the Preparation Guide.

Caricamento della Guida alla Preparazione su API Gemini...
✔️ Guida alla Preparazione caricata.



✔️ File API di Gemini puliti.
